In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Define paths to your dataset
DATASET_PATH = "Dataset/pest"
TRAIN_DIR = os.path.join(DATASET_PATH, "train")
TEST_DIR = os.path.join(DATASET_PATH, "test")

# Parameters
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32
EPOCHS = 30
NUM_CLASSES = len(os.listdir(TRAIN_DIR))  # Automatically determine number of classes

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2, 
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Only rescaling for validation/test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Load training data
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

# Load test data
test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Get class names
class_labels = list(train_generator.class_indices.keys())
print(f"Class labels: {class_labels}")
print(f"Total classes: {NUM_CLASSES}")

# Build CNN model
def build_cnn_model():
    model = Sequential([
        # First Convolutional Block
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Second Convolutional Block
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Third Convolutional Block
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Fully Connected Layers
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(NUM_CLASSES, activation='softmax')
    ])
    
    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Create model
model = build_cnn_model()
model.summary()

# Set up callbacks
checkpoint = ModelCheckpoint(
    'cnn_pest_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=0.00001,
    verbose=1
)

callbacks = [checkpoint, early_stopping, reduce_lr]

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=test_generator,
    validation_steps=test_generator.samples // BATCH_SIZE,
    callbacks=callbacks
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")

# Save the model
model.save('cnn_pest_model.h5')
print("Model saved as 'cnn_pest_model.h5'")

# Plot training history
plt.figure(figsize=(12, 5))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.savefig('training_history.png')
plt.show()

# Save class labels for prediction
import json
with open('class_labels.json', 'w') as f:
    json.dump(class_labels, f)
print("Class labels saved as 'class_labels.json'")

Found 2700 images belonging to 9 classes.
Found 450 images belonging to 9 classes.
Class labels: ['aphids', 'armyworm', 'beetle', 'bollworm', 'grasshopper', 'mites', 'mosquito', 'sawfly', 'stem_borer']
Total classes: 9


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 224, 224, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 112, 112, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 56, 56, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │    51,380,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 51,676,201 (197.13 MB)

 Trainable params: 51,674,281 (197.12 MB)

...

c:\Hemanth\Mini Project\venv\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 2: val_accuracy did not improve from 0.21429
84/84 ━━━━━━━━━━━━━━━━━━━━ 13s 115ms/step - accuracy: 0.3125 - loss: 2.0876 - val_accuracy: 0.2121 - val_loss: 2.3647 - learning_rate: 0.0010
Epoch 3/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2884 - loss: 2.3466
Epoch 3: val_accuracy did not improve from 0.21429
84/84 ━━━━━━━━━━━━━━━━━━━━ 301s 4s/step - accuracy: 0.2885 - loss: 2.3460 - val_accuracy: 0.1585 - val_loss: 2.6561 - learning_rate: 0.0010
Epoch 4/30
 1/84 ━━━━━━━━━━━━━━━━━━━━ 4:38 3s/step - accuracy: 0.4062 - loss: 2.2140
Epoch 4: val_accuracy did not improve from 0.21429
84/84 ━━━━━━━━━━━━━━━━━━━━ 13s 112ms/step - accuracy: 0.4062 - loss: 2.2140 - val_accuracy: 0.1540 - val_loss: 2.6630 - learning_rate: 0.0010
Epoch 5/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.3571 - loss: 2.0358
Epoch 5: val_accuracy did not improve from 0.21429
84/84 ━━━━━━━━━━━━━━━━━━━━ 313s 4s/step - accuracy: 0.3571 - loss: 2.0354 - val_accuracy: 0.1920 - val_loss: 2.1899 - le

84/84 ━━━━━━━━━━━━━━━━━━━━ 342s 4s/step - accuracy: 0.4007 - loss: 1.9278 - val_accuracy: 0.2924 - val_loss: 2.2955 - learning_rate: 0.0010
Epoch 8/30
 1/84 ━━━━━━━━━━━━━━━━━━━━ 3:19 2s/step - accuracy: 0.3438 - loss: 2.1501
Epoch 8: val_accuracy did not improve from 0.29241
84/84 ━━━━━━━━━━━━━━━━━━━━ 9s 79ms/step - accuracy: 0.3438 - loss: 2.1501 - val_accuracy: 0.2835 - val_loss: 2.3319 - learning_rate: 0.0010
Epoch 9/30
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4209 - loss: 1.7515
Epoch 9: val_accuracy did not improve from 0.29241
84/84 ━━━━━━━━━━━━━━━━━━━━ 273s 3s/step - accuracy: 0.4210 - loss: 1.7507 - val_accuracy: 0.2344 - val_loss: 2.4400 - learning_rate: 0.0010
Epoch 10/30
 1/84 ━━━━━━━━━━━━━━━━━━━━ 4:54 4s/step - accuracy: 0.3750 - loss: 1.8621
Epoch 10: val_accuracy did not improve from 0.29241
84/84 ━━━━━━━━━━━━━━━━━━━━ 11s 86ms/step - accuracy: 0.3750 - loss: 1.8621 - val_accuracy: 0.2656 - val_loss: 2.3482 - learning_rate: 0.0010
Epoch 11/30
84/84 ━━━━━━━━━━━━━

84/84 ━━━━━━━━━━━━━━━━━━━━ 250s 3s/step - accuracy: 0.4587 - loss: 1.5767 - val_accuracy: 0.5000 - val_loss: 1.3467 - learning_rate: 0.0010
Epoch 12/30
 1/84 ━━━━━━━━━━━━━━━━━━━━ 5:06 4s/step - accuracy: 0.4062 - loss: 1.4586
Epoch 12: val_accuracy improved from 0.50000 to 0.50893, saving model to cnn_pest_model.h5


84/84 ━━━━━━━━━━━━━━━━━━━━ 16s 145ms/step - accuracy: 0.4062 - loss: 1.4586 - val_accuracy: 0.5089 - val_loss: 1.3371 - learning_rate: 0.0010
Epoch 13/30
78/84 ━━━━━━━━━━━━━━━━━━━━ 16s 3s/step - accuracy: 0.4903 - loss: 1.4468